In [1]:
import sqlite3

In [4]:
# Create the SQLite database
conn = sqlite3.connect("smart_shop.db")
cursor = conn.cursor()

# Drop tables if they already exist for clean setup
cursor.executescript("""
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS interactions;
DROP TABLE IF EXISTS recommendations;
""")

# Create table: customers
cursor.execute("""
CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    gender TEXT,
    age INTEGER,
    location TEXT,
    browser_used TEXT,
    time_spent_on_site REAL
);
""")

# Create table: products
cursor.execute("""
CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price REAL,
    popularity_score REAL
);
""")

# Create table: interactions (customer browsing/purchase events)
cursor.execute("""
CREATE TABLE interactions (
    interaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id TEXT,
    product_id TEXT,
    interaction_type TEXT,
    timestamp TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
""")

# Create table: recommendations (for tracking past recommendations)
cursor.execute("""
CREATE TABLE recommendations (
    rec_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id TEXT,
    product_id TEXT,
    recommended_on TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
""")

conn.commit()

In [10]:
import pandas as pd
import sqlite3

# Load your CSVs
customer_df = pd.read_csv("customer_data_collection.csv", encoding="utf-8")
product_df = pd.read_csv("product_recommendation_data.csv", encoding="utf-8")


# Drop unnamed columns
customer_df = customer_df.loc[:, ~customer_df.columns.str.contains('^Unnamed')]
product_df = product_df.loc[:, ~product_df.columns.str.contains('^Unnamed')]

In [11]:
conn = sqlite3.connect("smart_shop.db")
cursor = conn.cursor()

# Drop old tables
cursor.executescript("""
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS products;
""")

# Create tables with matching schema
cursor.execute("""
CREATE TABLE customers (
    Customer_ID TEXT PRIMARY KEY,
    Age INTEGER,
    Gender TEXT,
    Location TEXT,
    Browsing_History TEXT,
    Purchase_History TEXT,
    Customer_Segment TEXT,
    Avg_Order_Value REAL,
    Holiday TEXT,
    Season TEXT
);
""")

cursor.execute("""
CREATE TABLE products (
    Product_ID TEXT PRIMARY KEY,
    Category TEXT,
    Subcategory TEXT,
    Price REAL,
    Brand TEXT,
    Average_Rating_of_Similar_Products REAL,
    Product_Rating REAL,
    Customer_Review_Sentiment_Score REAL,
    Holiday TEXT,
    Season TEXT,
    Geographical_Location TEXT,
    Similar_Product_List TEXT,
    Probability_of_Recommendation REAL
);
""")

# Insert data
customer_df.to_sql("customers", conn, if_exists="append", index=False)
product_df.to_sql("products", conn, if_exists="append", index=False)

conn.commit()
conn.close()
print("✅ Database setup complete!")

✅ Database setup complete!


In [12]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("smart_shop.db")
df = pd.read_sql_query("SELECT * FROM customers", conn)
df.head()

,Customer_ID,Age,Gender,Location,Browsing_History,Purchase_History,Customer_Segment,Avg_Order_Value,Holiday,Season
0,C1000,28,Female,Chennai,"['Books', 'Fashion']","['Biography', 'Jeans']",New Visitor,4806.99,No,Winter
1,C1001,27,Male,Delhi,"['Books', 'Fitness', 'Fashion']","['Biography', 'Resistance Bands', 'T-shirt']",Occasional Shopper,795.03,Yes,Autumn
2,C1002,34,Other,Chennai,['Electronics'],['Smartphone'],Occasional Shopper,1742.45,Yes,Summer
3,C1003,23,Male,Bangalore,['Home Decor'],['Wall Art'],Frequent Buyer,2023.16,No,Autumn
4,C1004,24,Other,Kolkata,"['Fashion', 'Home Decor']","['Shoes', 'Lamp']",Frequent Buyer,794.76,No,Winter
